In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install tensorflow-addons
import tensorflow_addons as tfa
from sklearn.model_selection import KFold, train_test_split
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.9 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
from pathlib import Path
import os.path
image_dir = Path('/content/gdrive/MyDrive/Hasil_Train/Dataset_128/0deg_LED_resized')
#image_dir = Path('/content/gdrive/MyDrive/Hasil_Train/Dataset_128/open_noLED_png_resized')

In [15]:
filepath = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
turbidity = pd.Series(filepath.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Turbidity').astype(np.float)

images = pd.concat([filepath, turbidity], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [16]:
def get_model():
    #Create model
    base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet", 
                                                  input_shape=(224, 224, 3), pooling='avg')
    out = base_model.output
    prediction = tf.keras.layers.Dense(1, activation="linear")(out)
    model = tf.keras.Model(inputs = base_model.input, outputs = prediction)
      
    return model

In [19]:
model = get_model()
model.load_weights(f"/content/gdrive/MyDrive/Hasil_Train/Training/0DEG-EfficientNetB0-Train0.9-CV-TL_8Fold/EfficientNet_8.h5")
opt = tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-6)
model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=['mae','mse', tfa.metrics.RSquare(name="R2")])

In [20]:
train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1)
Y = train_df[['Turbidity']]

In [21]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True
)

In [22]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    shuffle=False,
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    shuffle=False
)

Found 486 validated image filenames.
Found 55 validated image filenames.


In [23]:
train_pred = np.squeeze(model.predict(train_images))
train_true = train_images.labels
train_residuals = train_true - train_pred

test_pred = np.squeeze(model.predict(test_images))
test_true = test_images.labels
test_residuals = test_true - test_pred

In [ ]:
test_images.labels

In [24]:
from sklearn.metrics import mean_absolute_error as MAE
MAE_train = MAE(train_true,train_pred)
MAE_test = MAE(test_true,test_pred)
print(MAE_train)
print(MAE_test)

2.171496290918977
2.2766385798020794


In [25]:
from sklearn.metrics import r2_score as r2
r2_train = r2(train_true,train_pred)
r2_test = r2(test_true,test_pred)
print(r2_train)
print(r2_test)

0.9937931636274733
0.993425361809574


In [26]:
pd.DataFrame(test_residuals).to_csv("test_residuals_0deg_efnet.csv")

ABAIKAN

In [ ]:
pd.DataFrame(test_pred).to_csv("test_pred_90deg_efnet.csv")
pd.DataFrame(test_true).to_csv("y_test_90deg_efnet.csv")